In [ ]:
import yaml
from transformer_io.dataset import GeneIdTokenizer
from datamodules import MappedCollectionDataModule
from pathlib import Path
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_root = Path('/lustre/groups/ml01/projects/transformer-io/')
dataset_path = data_root / 'data/'
gene_mapping_path = data_root / 'gene_token_mapping.pkl'

gene_mapping = pd.read_pickle(gene_mapping_path).to_dict()

split = {'train': ['d8da613f-e681-4c69-b463-e94f5e66847f.h5ad', 
                   'a4116666-d295-4443-a174-0acc283f92c4.h5ad'
                   ],
          'val':  ['01ad3cd7-3929-4654-84c0-6db05bd5fd59.h5ad', 
                   'ae4f8ddd-cac9-4172-9681-2175da462f2e.h5ad']
          }

# yaml_file_path = "/home/icb/mojtaba.bahrami/projects/contrastive-transformer/conf/split/split_v1_large.yaml"
# with open(yaml_file_path, "r") as file:
#     split = yaml.safe_load(file)


dataset_args = {'train':{'n_tokens': 200, 'sub_sample_frac': 1.0}, 'val':{'n_tokens': 200, 'sub_sample_frac': 1.0}}
dataloader_args = {
  'train':
    {'shuffle': True,
    'num_workers': 1,
    'drop_last': True,
    'batch_size': 32,
    },
  'val':
    {'shuffle': False,
    'num_workers': 1,
    'drop_last': True,
    'batch_size': 32,
    },
}

datamodule_args = {
    'dataset_path': dataset_path,
    'split': split,
    'columns': [],
    'normalization': 'log1p',
    'dataset_kwargs': dataset_args,
    'dataloader_kwargs': dataloader_args,
    'tokenizer': GeneIdTokenizer(gene_mapping)
}

datamodule = MappedCollectionDataModule(**datamodule_args)

In [ ]:
train_loader = datamodule.train_dataloader() 
for i, batch in enumerate(train_loader):
    break
print(batch)

In [ ]:
import tqdm
for i, batch in tqdm(enumerate(train_loader)):
    pass